In [4]:
import cv2
import numpy as np
import time

In [5]:
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_gray = cv2.GaussianBlur(prev_gray, (21, 21), 0)

game_mode = "Red"

last_event_time = time.time()
event_interval = 5

In [6]:
while True:
    ret, frame = cap.read()
    key = cv2.waitKey(20) & 0xff
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    if game_mode == "Red":
        delta = cv2.absdiff(prev_gray, gray)
        thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        motion_map = np.zeros_like(frame)    
        for contour in contours:
            if cv2.contourArea(contour) > 100:
                (x, y, w, h) = cv2.boundingRect(contour)
                cv2.rectangle(motion_map, (x, y), (x + w, y + h), (0, 0, 255), -1)

        motion_overlay = cv2.addWeighted(frame, 0.5, motion_map, 0.5, 0)
        static_overlay = np.zeros_like(frame)
        frame = cv2.addWeighted(motion_overlay, 1, static_overlay, 1, 0)
        
    frame = cv2.putText(
        frame, f"{game_mode} color", (50, 50), cv2.FONT_ITALIC, 1, (
            0,
            255 if game_mode == "Green" else 0,
            255 if game_mode == "Red" else 0,
        ), 2)

    cv2.imshow("Motion Detection", frame)
    prev_gray = gray
    if key == 27: # Esc
        break
        
    current_time = time.time()
    if current_time - last_event_time >= event_interval:
        game_mode = "Green" if game_mode == "Red" else "Red"
        last_event_time = current_time


cap.release()
cv2.destroyAllWindows()
